## 1번 문제

In [5]:
import re

class Tokenizer():
    def __init__(self):
        self.word_dict = {'oov':0}
        self.fit_checker = False

    def preprocessing(self, sequences):
        
        result = []
        
        # 조건 1: 입력된 문장에 대해서 소문자로의 변환과 특수문자 제거를 수행합니다.
        for string in sequences:
            string = re.sub(r"[^a-zA-Z0-9 ]",'',string)
            string = string.lower()
            # 조건 2: 토큰화는 white space 단위로 수행합니다.
            list_spring = string.split()

            result.append(list_spring)

        return result

    def fit(self, sequences):
        self.fit_checker = False
        
        # 조건 1: 위에서 만든 preprocessing 함수를 이용하여 각 문장에 대해 토큰화를 수행합니다.
        token = self.preprocessing(sequences)
        ## num_token을 통해서 값을 하나하나 지정해 주어야 한다.##
        num_token = 1
        # 조건 2: 각각의 토큰을 정수 인덱싱 하기 위한 어휘 사전(self.word_dict)을 생성합니다.
        for string in token:
            for value in string:

                if value not in self.word_dict.keys():
                    self.word_dict[value] = num_token
                    num_token += 1
                else:
                    pass

        self.fit_checker = True

    def transform(self, sequences):

        result = []

        tokens = self.preprocessing(sequences)
        if self.fit_checker:
            for string in tokens:
                string_result = []
                for token in range(len(string)):
                    try:
                        value = self.word_dict[string[token]]
                        string_result.append(value)
                    except:
                        value = self.word_dict['oov']
                        string_result.append(value)
                result.append(string_result)
            return result
        else:
            raise Exception('Tokenizer instance is not fitted yet.')

    def fit_transform(self, sequences):
        self.fit(sequences)
        result = self.transform(sequences)
        return result

In [6]:
list_sequence = ['I have computer', 'I hate soccer', 'I Love Coffee', 'Good Weather']
list_sequence1 = ['I have keyboard', 'I hate excercise', 'hahahahohoho', 'I study AI']
Token = Tokenizer()
Token.fit(list_sequence)
print(Token.transform(list_sequence1))
print(Token.fit_transform(list_sequence))

[[1, 2, 0], [1, 4, 0], [0], [1, 0, 0]]
[[1, 2, 3], [1, 4, 5], [1, 6, 7], [8, 9]]


## 2번 문제

In [7]:
## TfidVectorizer 생성하기
import numpy as np

class TfidfVectorizer:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.fit_checker = False

    def fit(self, sequences):
        tokenized = self.tokenizer.fit_transform(sequences)
        '''
        문제 2-1
        '''
        num_string = len(tokenized)
        self.fit_token_unique = len(np.unique([j for i in tokenized for j in i]))
        self.fit_token_min = np.min([j for i in tokenized for j in i])

        results = []
        for i in tokenized:
            string = [0 for i in range(self.fit_token_unique)]
            for j in i:
                string[j - self.fit_token_min] += 1

            results.append(string)

        matrix_result = np.matrix(results).sum(axis = 0)
        self.IDF = np.log(num_string/(1 + matrix_result)).tolist()[0]
        self.fit_checker = True

    def transform(self, sequences):
        if self.fit_checker:
            tokenized = self.tokenizer.transform(sequences)

            tf_results = []
            for i in tokenized:
                string = [0 for i in range(self.fit_token_unique)]
                for j in i:
                    string[j - self.fit_token_min] += 1.0

                tf_results.append(string)

            self.tfidf_matrix = []
            for string in tf_results:
                tfidf_list = [i*j for i,j in zip(string, self.IDF)]
                self.tfidf_matrix.append(tfidf_list)

            return self.tfidf_matrix
        else:
            raise Exception('TfidfVectorizer instance is not fitted yet.')

    def fit_transform(self, sequences):
        self.fit(sequences)
        return self.transform(sequences)

In [8]:
model = TfidfVectorizer(Tokenizer())
model.fit(list_sequence)
model.transform(list_sequence)
model.fit_transform(list_sequence)

[[0.0, 0.6931471805599453, 0.6931471805599453, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.6931471805599453, 0.6931471805599453, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.6931471805599453, 0.6931471805599453, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6931471805599453, 0.6931471805599453]]